In [44]:
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
#https://towardsdatascience.com/anova-tukey-test-in-python-b3082b6e6bda
experiment_names = ['owl_exp_pure',
                    'owl_exp_mixed',
                    'strike_exp_pure',
                    'strike_exp_mixed',
                    'energy_gain_pure',
                    'energy_gain_mixed',
                    ]

def query_exp_data(label):
    total_exp_data=[]
    for i in range(1,7):
        file_path_totals = '/home/mremington/Documents/uumarrty_exps/{}/exp{}/Data/totals.csv'.format(label,i)
        temp_exp_total = pd.read_csv(file_path_totals,header = 0, index_col=None)
        temp_exp_total['experiment_name'] = [label for i in range(len(temp_exp_total))]
        total_exp_data.append(temp_exp_total)
    total_exp_data = pd.concat(total_exp_data, ignore_index=False)
    return total_exp_data

def column_labels():
    orgs=['krat','snake']
    column_labels=[]
    column_labels.append('id')
    for org in orgs:
        for i in range(1,7):
            label='{}_exp_{}'.format(org,i)
            column_labels.append(label)
    return column_labels

def append_string(fp,string):
    with open(fp, 'a') as f:
        f.write(string+"\n")

In [54]:
experiment_names = ['owl_exp_pure',
                    'owl_exp_mixed',
                    'strike_exp_pure',
                    'strike_exp_mixed',
                    'energy_gain_pure',
                    'energy_gain_mixed',
                    ]
txt_file_path='anova_results_03102022.txt'

with open(txt_file_path, 'w') as f:
    pass

for exp in experiment_names:
    intro = "Anova Results for Experiment: {}".format(exp)
    append_string(fp=txt_file_path,string=intro)
    append_string(fp=txt_file_path,
            string="".join(["_" for i in range(len(intro))]))
    append_string(fp=txt_file_path,string='')
    append_string(fp=txt_file_path,string='')
    data = query_exp_data(label=exp)
    # Anova
    pivot_anova = data.pivot_table(index=["sim_number"], 
                            columns=['data_type','experiment'], 
                            values='mean_bush_pref')
    pivot_anova = pivot_anova.reset_index()
    pivot_anova.columns = column_labels()
    #krats
    append_string(fp=txt_file_path,string='Krat')
    krat_fvalue, krat_pvalue = stats.f_oneway(pivot_anova['krat_exp_1'],
                                              pivot_anova['krat_exp_2'],
                                              pivot_anova['krat_exp_3'],
                                              pivot_anova['krat_exp_4'],
                                              pivot_anova['krat_exp_5'],
                                              pivot_anova['krat_exp_6'],)
    append_string(fp=txt_file_path,string='F Value: {}, P Value {}'.format(krat_fvalue, krat_pvalue))
    append_string(fp=txt_file_path,string='')
    #Snakes
    append_string(fp=txt_file_path,string='Snake')
    snake_fvalue, snake_pvalue = stats.f_oneway(pivot_anova['snake_exp_1'],
                                                pivot_anova['snake_exp_2'],
                                                pivot_anova['snake_exp_3'],
                                                pivot_anova['snake_exp_4'],
                                                pivot_anova['snake_exp_5'],
                                                pivot_anova['snake_exp_6'],)
    append_string(fp=txt_file_path,string='F Value: {}, P Value {}'.format(snake_fvalue, snake_pvalue))
    append_string(fp=txt_file_path,string='')
    #Tukey
    intro = "Tukey Results for Experiment: {}".format(exp)
    append_string(fp=txt_file_path,string=intro)
    pivot_tukey = data.pivot_table(index=["experiment","sim_id"], 
                            columns=['data_type'], 
                            values='mean_bush_pref')
    pivot_tukey = pivot_tukey.reset_index()
    pivot_tukey.columns=['experiment','sim_id','krat','snake']
    m_comp = pairwise_tukeyhsd(endog=pivot_tukey['krat'], groups=pivot_tukey['experiment'], alpha=0.05)
    append_string(fp=txt_file_path,string=str(m_comp))
    append_string(fp=txt_file_path,string="")
    append_string(fp=txt_file_path,string="")